In [1]:
 
import os

In [3]:
%pwd


'd:\\kidney_proj\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\kidney_proj'

In [6]:
%pip install -q dagshub mlflow



Note: you may need to restart the kernel to use updated packages.


In [7]:
import dagshub
dagshub.init(repo_owner='anshii2019', repo_name='kidney_disease_classification_deep_learning_project', mlflow=True)

Accessing as anshii2019

Initialized MLflow to track repo "anshii2019/kidney_disease_classification_deep_learning_project"

Repository anshii2019/kidney_disease_classification_deep_learning_project initialized!

In [8]:
import mlflow
with mlflow.start_run():
  # Your training code here...
  mlflow.log_metric('accuracy', 42)
  mlflow.log_param('Param name', 'Value')

2025/02/28 15:17:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run polite-grub-241 at: https://dagshub.com/anshii2019/kidney_disease_classification_deep_learning_project.mlflow/#/experiments/0/runs/f8a27be49e48451f811ac8e192874b47.
2025/02/28 15:17:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/anshii2019/kidney_disease_classification_deep_learning_project.mlflow/#/experiments/0.


In [10]:
mlflow.autolog()

In [11]:
import tensorflow as tf


2025/02/28 15:18:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/02/28 15:18:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


In [12]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [17]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [18]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [20]:

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/anshii2019/kidney_disease_classification_deep_learning_project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [21]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [23]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")



In [24]:

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-02-28 15:18:39,857:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-02-28 15:18:39,864:INFO:common:yaml file: params.yaml loaded successfully]
[2025-02-28 15:18:39,867:INFO:common:created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 33s 3s/step - loss: 9.7006 - accuracy: 0.5180
[2025-02-28 15:19:14,437:INFO:common:json file saved at: scores.json]


2025/02/28 15:19:15 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\Anshika\AppData\Local\Temp\tmpkhfiv_ud\model\data\model\assets
[2025-02-28 15:19:18,529:INFO:builder_impl:Assets written to: C:\Users\Anshika\AppData\Local\Temp\tmpkhfiv_ud\model\data\model\assets]


2025/02/28 15:19:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/02/28 15:20:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
2025/02/28 15:20:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run loud-finch-976 at: https://dagshub.com/anshii2019/kidney_disease_classification_deep_learning_project.mlflow/#/experiments/0/runs/9ed820981caa4bc6bfa2d481ca6b7052.
2025/02/28 15:20:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/anshii2019/kidney_disease_classification_deep_learning_project.mlflow/#/experiments/0.
